## Cutting diaObjects and diaSources into usable Tables

We are trying to access a bunch of tables in parquet files. In order to access this, we read the parquet file and convert the data in a Pandas DataFrame. After that, we must filter out most of the objects, because we only want to test 2. Therefore we take the top 2 objects from the table. Then we find the diaSourceIds in the association table. I then created a function to create a new source table with only the sources associated with the object.

In [1]:
import pandas as pd

object_table_link = "/global/cscratch1/sd/bos0109/results/association_fix_test/diaObject_table.pqt"
object_table = pd.read_parquet(object_table_link, engine='pyarrow')

assoc_table_link = "/global/cscratch1/sd/bos0109/results/association_fix_test/assoc_table.pqt"
assoc_table = pd.read_parquet(assoc_table_link, engine='pyarrow')

source_table_link = "/global/cscratch1/sd/bos0109/results/association_fix_test/full_table_diasrc.pqt"
source_table = pd.read_parquet(source_table_link, engine='pyarrow')

In [2]:
object_table = object_table.head(2)
assoc_table = assoc_table.head(2)

In [3]:
#Helper function for create_source_table
def _find_row(object_number):
    lst = []
    for srcId in assoc_table["diaSrcIds"][object_number]:
        i = 0
        while srcId != source_table["id"][i]:
            i += 1
    
        lst.append(i)
    
    return lst

def create_source_table(object_number):
    row_numbers = _find_row(object_number)
    ids_ = []
    for row in row_numbers:
        ids_.append(source_table['id'][row])
    
    #Now i have a whole list of ids i need to grab
        
    return source_table.query(f'id in {ids_}')

In [4]:
source_table_0 = create_source_table(0)
source_table_1 = create_source_table(1)
#The second one takes a while because there are a lot of sources for 1 object

In [5]:
object_table
source_table_0

,id,coord_ra,coord_dec,parent,flags_negative,base_NaiveCentroid_x,base_NaiveCentroid_y,base_NaiveCentroid_flag,base_NaiveCentroid_flag_noCounts,base_NaiveCentroid_flag_edge,...,ip_diffim_forced_PsfFlux_mag,ip_diffim_forced_PsfFlux_fluxErr,ip_diffim_forced_PsfFlux_magErr,visit_n,filter,raft,sensor,detector,mjd,airmass
9891,18448170879025282,1.010832,-0.542038,0,False,52.887738,1744.243569,False,False,False,...,16.876049,768.149300,0.001293,274899,r,R41,S20,168,59965.122153,1.758695
34110,16162841866272830,1.010834,-0.542038,0,False,3677.171224,2748.173670,False,False,False,...,15.174869,2456.892377,0.000863,240845,z,R24,S11,112,59912.055146,1.307687
34980,14872201442361435,1.010833,-0.542038,0,False,881.348382,1950.788068,False,False,False,...,19.185683,230.093050,0.003250,221613,g,R14,S22,71,59879.201456,1.301180
131986,458095239102546,1.010832,-0.542037,0,False,3590.351135,3224.025140,False,False,False,...,16.976622,858.515057,0.001585,6826,i,R33,S21,151,59589.201274,2.227943
170894,28755749731565710,1.010832,-0.542039,0,False,3118.189572,2225.055282,False,False,False,...,16.601859,1020.137408,0.001334,428494,i,R13,S22,62,60193.323393,1.320216
183650,14869513061269601,1.010832,-0.542039,0,False,1556.270830,369.645428,False,False,False,...,19.632707,173.741807,0.003704,221573,g,R02,S02,11,59879.181631,1.325329
242851,29348251006140602,1.010831,-0.542038,0,False,1976.036553,3841.074411,False,False,False,...,16.858078,905.405085,0.001499,437323,i,R03,S01,19,60207.235930,1.329430
245141,11713786981384289,1.010831,-0.542040,0,False,1393.007469,1041.085133,False,False,False,...,17.492418,651.757529,0.001935,174549,i,R10,S01,28,59810.356861,1.329904
327404,47998744506925274,1.010832,-0.542037,0,False,2567.330895,1646.376957,False,False,False,...,18.478295,314.873471,0.002318,715237,r,R10,S02,29,60635.101358,1.335870
338045,34360012709036169,1.010832,-0.542040,0,False,170.976577,3138.070789,False,False,False,...,16.363918,1158.375437,0.001217,512004,i,R21,S21,88,60330.110857,1.732427


In [6]:
assoc_table

,diaObjectId,diaSrcIds
0,20407348128382977,"[458095239102546, 11713786981384289, 148695130..."
1,20407348128382978,"[458095239102549, 10702459085258959, 107025922..."
